In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/instacart-market-basket-analysis/products.csv
/kaggle/input/instacart-market-basket-analysis/departments.csv
/kaggle/input/instacart-market-basket-analysis/sample_submission.csv
/kaggle/input/instacart-market-basket-analysis/order_products__prior.csv
/kaggle/input/instacart-market-basket-analysis/order_products__train.csv
/kaggle/input/instacart-market-basket-analysis/aisles.csv
/kaggle/input/instacart-market-basket-analysis/orders.csv


In [3]:

from __future__ import absolute_import, division, print_function, unicode_literals
try:
  %tensorflow_version 2.x
except Exception:
  pass
#make feature based dataset
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from IPython.display import clear_output
import IPython
import plotly.express as px
import plotly.offline as pyo
import plotly.graph_objects as go

#pyo.init_notebook_mode()



In [4]:
#reading data
orders=pd.read_csv("../input/instacart-market-basket-analysis/orders.csv")

departments=pd.read_csv("../input/instacart-market-basket-analysis/departments.csv")
aisles=pd.read_csv("../input/instacart-market-basket-analysis/aisles.csv")
products=pd.read_csv("../input/instacart-market-basket-analysis/products.csv")
order_product_train=pd.read_csv("../input/instacart-market-basket-analysis/order_products__train.csv")
order_product_prior=pd.read_csv("../input/instacart-market-basket-analysis/order_products__prior.csv")
print("[Out]:Files Successfully Loaded in frames")

[Out]:Files Successfully Loaded in frames


In [5]:
#showing data
#printing 5 top row from each file
print("\norder:")
print(orders.head(5))
print("\ndepartments:")
print(departments.head(5))

print("\naisles:")

print(aisles.head(5))

print("\nproducts:")
print(products.head(5))

print("\n order_product_train")

print(order_product_train.head(5))
print("\n order_product_pripor:")

print(order_product_prior.head(5))




order:
   order_id  user_id eval_set  order_number  order_dow  order_hour_of_day  \
0   2539329        1    prior             1          2                  8   
1   2398795        1    prior             2          3                  7   
2    473747        1    prior             3          3                 12   
3   2254736        1    prior             4          4                  7   
4    431534        1    prior             5          4                 15   

   days_since_prior_order  
0                     NaN  
1                    15.0  
2                    21.0  
3                    29.0  
4                    28.0  

departments:
   department_id department
0              1     frozen
1              2      other
2              3     bakery
3              4    produce
4              5    alcohol

aisles:
   aisle_id                       aisle
0         1       prepared soups salads
1         2           specialty cheeses
2         3         energy granola bars
3         

In [6]:
#shape of dataframes
def shape_info():
    print("[OUT]:orders shape:",orders.shape)
    print("[OUT]:departments shape:",departments.shape)
    print("[OUT]:aisles shape",aisles.shape)
    print("[OUT]:products shape",aisles.shape)
    print("[OUT]:order product train shape",order_product_train.shape)
    print("[OUT]:order product prior shape",order_product_prior.shape)
shape_info()

[OUT]:orders shape: (3421083, 7)
[OUT]:departments shape: (21, 2)
[OUT]:aisles shape (134, 2)
[OUT]:products shape (134, 2)
[OUT]:order product train shape (1384617, 4)
[OUT]:order product prior shape (32434489, 4)


In [7]:
#data preprocessing 

def missing_summary():
    order_miss=orders.isnull().sum()
    print("[OUT]:Missing data summary for orders: \n",pd.concat([order_miss,order_miss/orders.isnull().count()],keys=['Total Miss','Percentage Miss'],axis=1))
    
    depa_miss=departments.isnull().sum()
    print("\n\n[OUT]:Missing data summary for departments:\n",pd.concat([depa_miss,depa_miss/departments.isnull().count()],keys=['Total Miss','Percentage Miss'],axis=1))
    
    aisle_miss=aisles.isnull().sum()
    print("\n\n[OUT]:Missing data summary for aisles:\n",pd.concat([aisle_miss,aisle_miss/aisles.isnull().count()],keys=['Total Miss','Percentage Miss'],axis=1))
    
    prod_miss=products.isnull().sum()
    print("\n\n[OUT]:Missing values summary for departments:\n",pd.concat([prod_miss,prod_miss/products.isnull().count()],keys=['Total Miss','Percentage Miss'],axis=1))
    
    op_train_miss=order_product_train.isnull().sum()
    print("\n\n[OUT]:Missing values summary for departments:\n",pd.concat([op_train_miss,op_train_miss/order_product_train.isnull().count()],keys=['Total Miss','Percentage Miss'],axis=1))
    
    op_prior_miss=order_product_prior.isnull().sum()
    print("\n\n[OUT]:Missing values summary for departments:\n",pd.concat([op_prior_miss,op_prior_miss/order_product_prior.isnull().count()],keys=['Total Miss','Percentage Miss'],axis=1))
    
    
missing_summary()
    

[OUT]:Missing data summary for orders: 
                         Total Miss  Percentage Miss
order_id                         0         0.000000
user_id                          0         0.000000
eval_set                         0         0.000000
order_number                     0         0.000000
order_dow                        0         0.000000
order_hour_of_day                0         0.000000
days_since_prior_order      206209         0.060276


[OUT]:Missing data summary for departments:
                Total Miss  Percentage Miss
department_id           0              0.0
department              0              0.0


[OUT]:Missing data summary for aisles:
           Total Miss  Percentage Miss
aisle_id           0              0.0
aisle              0              0.0


[OUT]:Missing values summary for departments:
                Total Miss  Percentage Miss
product_id              0              0.0
product_name            0              0.0
aisle_id                0        

In [8]:
#define preprocessing

def clean(orders):
    print("\n\n[OUT]: Cleaning Starts:")
    orders=orders[orders['days_since_prior_order'].notnull()]
    print("\n[OUT]: Cleaning Done.")
clean(orders)

print("\n[OUT]:Shapes After Clean:\n")
shape_info()






[OUT]: Cleaning Starts:

[OUT]: Cleaning Done.

[OUT]:Shapes After Clean:

[OUT]:orders shape: (3421083, 7)
[OUT]:departments shape: (21, 2)
[OUT]:aisles shape (134, 2)
[OUT]:products shape (134, 2)
[OUT]:order product train shape (1384617, 4)
[OUT]:order product prior shape (32434489, 4)


In [9]:

class data_visual_order():
    
    def eval_set(self):  
       
        distribution_count=orders['eval_set'].value_counts().rename_axis('Label').reset_index(name='Order_Count')
        print("\n[Out]: Count of Prior and Train data in orders : \n",distribution_count)
        fig = px.bar(distribution_count, x='Label', y='Order_Count',title='Distribution of Data as Train,Test and Prior',color='Label')
        fig.show()

    def order_dow(self):
        day_of_week_count=orders['order_dow'].value_counts().rename_axis('Week_Day').reset_index(name='Order_Count')
        week=['Sunday','Monday','Tuesday','Wednesday','Thursday','Friday','Saturday']
        day_of_week_count['Day_Name']=week
        fig = px.bar(day_of_week_count, x='Day_Name', y='Order_Count',title='Count of Order Per Day of Week',color='Day_Name')
        fig.show()
    def order_hour_of_day(self):
        hour_day=orders['order_hour_of_day'].value_counts().rename_axis('Hour_of_Day').reset_index(name='Order_Count')
        fig = px.bar(hour_day, x='Hour_of_Day', y='Order_Count',title='Count of Orders per Hour of Day',color='Hour_of_Day')
        fig.show()
        
    def per_week_day_hour(self):
        for i in range(7):
            week_data=orders[orders['order_dow']==i]
            week=['Sunday','Monday','Tuesday','Wednesday','Thursday','Friday','Saturday']
            title='On '+week[i]+' Count of Order Per hour'
            week_day_hour=week_data['order_hour_of_day'].value_counts().rename_axis('Hour_of_Day').reset_index(name='Order_Count')
            fig = px.bar(week_day_hour, x='Hour_of_Day', y='Order_Count',title=title,color='Hour_of_Day')
            fig.show()
    def day_hour_combined(self):
        day_hour_order = orders.groupby(["order_dow", "order_hour_of_day"])["order_number"].aggregate("count").reset_index()
        
        day_hour_order= day_hour_order.pivot('order_dow', 'order_hour_of_day', 'order_number')
        week=['Sunday','Monday','Tuesday','Wednesday','Thursday','Friday','Saturday']
        #print(np.array(day_hour_order))
        fig=go.Figure(data=go.Heatmap(z=np.array(day_hour_order),y=week))
        fig.show()

    def days_prior_order(self):
        prior_day = orders['days_since_prior_order'].value_counts().rename_axis('Days_Since_Prior_Order').reset_index(name='Order_Count')
        fig = px.bar(prior_day, x='Days_Since_Prior_Order', y='Order_Count',title='Count of order w.r.t. Days Prior to order',color='Days_Since_Prior_Order')
        fig.show()

or_vis=data_visual_order()
or_vis.eval_set()
or_vis.order_dow()
or_vis.order_hour_of_day()
or_vis.per_week_day_hour()
or_vis.day_hour_combined()
or_vis.days_prior_order()



[Out]: Count of Prior and Train data in orders : 
    Label  Order_Count
0  prior      3214874
1  train       131209
2   test        75000


In [10]:
class reorderd_relationship_visual():
    def __init__(self):
        self.merg_order_prior = pd.merge(order_product_prior, orders, on='order_id', how='left')
        self.merged_all= pd.merge(order_product_prior, products, on='product_id', how='left')
        self.merged_all= pd.merge(self.merged_all, aisles, on='aisle_id', how='left')
        self.merged_all=pd.merge(self.merged_all, departments, on='department_id', how='left')

    def reorderd_stats(self):
        reord_count=order_product_prior['reordered'].value_counts().rename_axis('Label').reset_index(name='Order_Count')
        reord_count['label_name']=['reordered','not reordered']
        fig = px.bar(reord_count, x='label_name', y='Order_Count',title='Count of Orders Which Were reordered',color='label_name')
        fig.show()
        
    def reorder_week(self):
        aggr_dow_reord = self.merg_order_prior.groupby(["order_dow"])["reordered"].aggregate("count").reset_index()

        aggr_dow_reord['Day_of_Week']=['Sunday','Monday','Tuesday','Wednesday','Thursday','Friday','Saturday']
        aggr_dow_reord.columns=['order_dow','Count of reordered','Day_of_Week']
        fig = px.bar(aggr_dow_reord, x='Day_of_Week', y='Count of reordered',title='Count of reorders in day of week',color='Day_of_Week')
        fig.show()
        
    def reorder_hour(self):
        aggr_hur_reord = self.merg_order_prior.groupby(["order_hour_of_day"])["reordered"].aggregate("count").reset_index()
        
        aggr_hur_reord.columns=['order_hour_of_day','Count of reordered']
        fig = px.bar(aggr_hur_reord, x='order_hour_of_day', y='Count of reordered',title='Count of reorders per hour in a day',color='order_hour_of_day')
        fig.show()
    def product_reorder(self):
        reord_products = self.merged_all['product_name'].value_counts().rename_axis('Product Name').reset_index(name="Reorder Count").head(10)
        fig = px.bar(reord_products, x='Product Name', y='Reorder Count',title='Count of reorders w.r.t Product Name',color='Product Name')
        fig.show()
    def aisle_reorder(self):
        reord_aisle = self.merged_all['aisle'].value_counts().rename_axis('Aisle Name').reset_index(name="Reorder Count").head(10)
        fig = px.bar(reord_aisle, x='Aisle Name', y='Reorder Count',title='Count of reorders w.r.t Aisle',color='Aisle Name')
        fig.show()
    def depart_reorder(self):
        reord_depa = self.merged_all['department'].value_counts().rename_axis('Department Name').reset_index(name="Reorder Count")
        fig = px.bar(reord_depa, x='Department Name', y='Reorder Count',title='Count of reorders w.r.t Departments',color='Department Name')
        fig.show()
re_vis=reorderd_relationship_visual()
re_vis.reorderd_stats()
re_vis.reorder_week()
re_vis.reorder_hour()
re_vis.product_reorder()
re_vis.aisle_reorder()
re_vis.depart_reorder()


In [11]:

#merging all tables

all_merged = pd.merge(order_product_train, orders, on='order_id', how='left')
all_merged = pd.merge(all_merged, products, on='product_id', how='left')
all_merged= pd.merge(all_merged, departments, on='department_id', how='left')
all_merged.head()




,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id,department
0,1,49302,1,1,112108,train,4,4,10,9.0,Bulgarian Yogurt,120,16,dairy eggs
1,1,11109,2,1,112108,train,4,4,10,9.0,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,16,dairy eggs
2,1,10246,3,0,112108,train,4,4,10,9.0,Organic Celery Hearts,83,4,produce
3,1,49683,4,0,112108,train,4,4,10,9.0,Cucumber Kirby,83,4,produce
4,1,43633,5,1,112108,train,4,4,10,9.0,Lightly Smoked Sardines in Olive Oil,95,15,canned goods


In [12]:
#all dataset as train
train=all_merged.copy()
test=train['reordered']
del train['reordered']
del  train['user_id']
del train['product_name']
del train['department']
del train['eval_set']
del train['add_to_cart_order']

In [13]:
#split data set as train and test
X_train, X_test, y_train, y_test = train_test_split(train, test, test_size=0.30, random_state=7)
print("[OUT]:length of trainig set: ",len(X_train))
print("[OUT]:length of test set: ",len(X_test))

[OUT]:length of trainig set:  969231
[OUT]:length of test set:  415386


In [14]:
#feaure colum
print("\n[Out] Making Feature Column:")

fc = tf.feature_column
Categorical_col = ['order_dow', 'order_hour_of_day','aisle_id', 'department_id','product_id']
Numerical_col = ['order_id', 'order_number','days_since_prior_order']

def one_hot_cat_column(feature_name, vocab):
  return fc.indicator_column(fc.categorical_column_with_vocabulary_list(feature_name,vocab))
feature_column = []
for feature_name in Categorical_col:
  # Need to one-hot encode categorical features.
  vocabulary = all_merged[feature_name].unique()
  feature_column.append(one_hot_cat_column(feature_name, vocabulary))

for feature_name in Numerical_col:
  feature_column.append(fc.numeric_column(feature_name,dtype=tf.float32))

print("\n[Out] Making of  Feature Column done.")


[Out] Making Feature Column:

[Out] Making of  Feature Column done.


In [16]:
#mak
size_of_batch = len(y_train)

def make_input_fn(X, y, n_epochs=None, shuffle=True):
  def input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((X.to_dict(orient='list'), y))
    if shuffle:
      dataset = dataset.shuffle(size_of_batch)
    # For training, cycle thru dataset as many times as need (n_epochs=None).
    dataset = (dataset.repeat(n_epochs).batch(size_of_batch))
    return dataset
  return input_fn

# Training and evaluation input functions.
train_suffle = make_input_fn(X_train,y_train)
test_shuffle =make_input_fn(X_test,y_test,shuffle=False, n_epochs=1)

In [ ]:

#boosted classifier
params = {
  'n_trees': 1,
  'max_depth': 1,
  'n_batches_per_layer': 1000,
  'center_bias': True
}
est = tf.estimator.BoostedTreesClassifier(feature_column, **params)
# Train model.
print("\n[OUT]:model trainig starts:")
est.train(train_suffle, max_steps=1)
print("\n[OUT]:model trainig end.")

print("\n[OUT]:model evaluation starts:")
results = est.evaluate(test_shuffle)
print("\n[OUT]:model evaluation end:")
clear_output()
pd.Series(results).to_frame()


[OUT]:model trainig starts:
